In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
import keras as k
from tensorflow.keras.layers import Lambda, Cropping2D, Dense, GlobalAveragePooling2D, Flatten, ZeroPadding2D, Conv2D, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential, Model
import numpy as np

In [ ]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.99)
gpu_options.allow_growth = True

sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

## Data Loader

In [ ]:
from DataGenerator2 import DataGenerator2
from NeptuneMonitor import NeptuneMonitor

## Layer to grayscale converter

In [ ]:
def grayscale_converter(x):
    return (0.21 * x[:,:,:,:1]) + (0.72 * x[:,:,:,1:2]) + (0.07 * x[:,:,:,-1:])

def saturation_converter(x):
    hsv = tf.image.rgb_to_hsv(x)
    return hsv[: , : , : , :1: ]
    #return cv2.cvtColor(x, cv2.COLOR_RGB2HSV)[:,:,1]

def resize_converter(x):
    return tf.keras.backend.resize_images(x, height_factor=0.5, width_factor=0.5, data_format="channels_last")

## Model preparation - own

In [ ]:
# Lambda(resize_converter, name='resize'),
# Flatten(name='flatten'),

model = Sequential([
    Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3), name='normalize'),
    Cropping2D(cropping=((65,25), (0,0)), name='cropping_65_25'),
    BatchNormalization(),
    Conv2D(24,(5,5), padding='valid', activation='relu'),
    BatchNormalization(),
    Conv2D(36,(5,5), padding='valid', activation='relu'),
    BatchNormalization(),
    Conv2D(48,(5,5), padding='valid', activation='relu'),
    BatchNormalization(),
    Conv2D(64,(3,3), padding='valid', activation='relu'),
    BatchNormalization(),
    Conv2D(64,(3,3), padding='valid', activation='relu'),
    Flatten(),
    Dense(100, activation='linear', name='dense-100'),
    Dense(50, activation='linear', name='dense-50'),
    Dense(10, activation='linear', name='dense-10'),
    Dense(1, activation='linear', name='dense-1')
])
model.summary()

In [ ]:
epochs = 120
patience = 10
batch_size = 32
learn_rate = 0.001

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=learn_rate)

In [ ]:
model.compile(loss='mse', optimizer=optimizer)

In [ ]:
my_generator = DataGenerator2('./my_data/', epochs=epochs, batch_size=batch_size, balance=False, debug=False)
udacity_generator = DataGenerator2('./udacity_data/', epochs=epochs, batch_size=batch_size, balance=False, debug=False)
my_generator.balance = True
udacity_generator.balance = True
generator = my_generator.merge(udacity_generator)

train, valid = generator.split(factor = 0.2)

In [ ]:
trainable_layers = None
params = {'epochs': epochs, 'lr':learn_rate, 'batch_size':batch_size, 'trainable': trainable_layers, 'balanced_dataset': True}
logger = NeptuneMonitor('own_nvidia', 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vbmVwdHVuZS5pbnRpdmUub3JnIiwiYXBpX2tleSI6IjAzZmMyZjBlLWY2ODQtNDQ1Yi1hNjU5LTAwMjNmNTFhMDc0YyJ9', 'grzegorz.tyminski/Behavioral-Clonning', params)

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.3, patience=patience, min_lr=1e-6, verbose=1)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=patience+1)
fit_result = model.fit_generator(train,
                                 steps_per_epoch=train.batches,
                                 epochs=train.epochs,
                                 verbose=1,
                                 validation_data=valid,
                                 validation_steps=valid.batches,
                                 callbacks=[reduce_lr, early_stop, logger])

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(fit_result.history['loss'])

In [ ]:
for i,v in enumerate(fit_result.history['loss']):
    print(f"EPOCH {i+1}:{v:0.5f}")